## Libraries executar antes do load

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

## Preprocessing the Training Set executar antes do load

In [6]:
# Cria um gerador de imagens para dados de treinamento
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Normaliza os valores dos pixels para o intervalo [0, 1]
    shear_range=0.2,        # Aplica transformações de cisalhamento em até 20% nos eixos
    zoom_range=0.2,         # Aplica zoom aleatório nas imagens (aumenta/diminui em até 20%)
    horizontal_flip=True    # Permite espelhamento horizontal aleatório das imagens
)

# Cria um conjunto de dados de treinamento a partir de imagens organizadas em diretórios
train_set = train_datagen.flow_from_directory(
    'dataset/training_set/',  # Caminho para o diretório que contém as imagens de treinamento
    target_size=(64, 64),     # Redimensiona todas as imagens para 64x64 pixels
    batch_size=32,            # Número de imagens a serem processadas por vez (lote)
    class_mode='binary'       # Define o tipo de classificação como binária (duas classes: 0 ou 1)
)

Found 8000 images belonging to 2 classes.


## Preprocessing the Test Set executar antes do load

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = train_datagen.flow_from_directory(
    'dataset/test_set/',
    target_size=(64, 64),     
    batch_size=32,           
    class_mode='binary'       
)

Found 2000 images belonging to 2 classes.


## Convolution Pooling Flattening

In [11]:
cnn = tf.keras.models.Sequential([
    
    # Define a entrada do modelo: imagens de 64x64 pixels com 3 canais (RGB)
    tf.keras.layers.Input(shape=(64, 64, 3)),
    
    # Primeira camada convolucional com 32 filtros e tamanho do kernel 3x3, ativação ReLU
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    
    # Camada de pooling para reduzir a dimensionalidade espacial (downsampling) com janela 2x2
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    
    # Segunda camada convolucional com 32 filtros e tamanho do kernel 3x3, ativação ReLU
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    
    # Outra camada de pooling para reduzir ainda mais a dimensionalidade espacial
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    
    # Camada de achatamento (flatten) para transformar os dados 2D em um vetor 1D
    tf.keras.layers.Flatten(),
    
    # Camada totalmente conectada com 128 unidades e ativação ReLU
    tf.keras.layers.Dense(units=128, activation='relu'),
    
    # Camada de saída com 1 unidade e ativação sigmoide para classificação binária
    tf.keras.layers.Dense(units=1, activation='sigmoid')

])

## Traning the CNN

In [13]:
cnn.compile( optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
cnn.fit(x = train_set, validation_data = test_set, epochs = 25)

C:\Users\Gabriel\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.5476 - loss: 0.6845

C:\Users\Gabriel\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 66s 256ms/step - accuracy: 0.5478 - loss: 0.6844 - val_accuracy: 0.5810 - val_loss: 0.6685
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 103ms/step - accuracy: 0.6789 - loss: 0.5879 - val_accuracy: 0.7290 - val_loss: 0.5414
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 102ms/step - accuracy: 0.7223 - loss: 0.5464 - val_accuracy: 0.7395 - val_loss: 0.5281
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 104ms/step - accuracy: 0.7368 - loss: 0.5129 - val_accuracy: 0.7520 - val_loss: 0.5199
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 99ms/step - accuracy: 0.7520 - loss: 0.5013 - val_accuracy: 0.7555 - val_loss: 0.5015
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 98ms/step - accuracy: 0.7831 - loss: 0.4597 - val_accuracy: 0.7645 - val_loss: 0.4915
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 101ms/step - accuracy: 0.7874 - loss: 0.4527 - val_accuracy: 0.7810 - val_loss: 0.4778
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 99ms/step - accuracy: 0.7883 - loss: 0.4451 - val_ac

In [35]:
# cnn.save('modelo_gato_cachorro.keras')

## Single Prediction

In [10]:
cnn = load_model('modelo_gato_cachorro.keras', compile=False)

In [12]:
# 1. Carrega a imagem e redimensiona para 64x64 pixels (tamanho esperado pelo modelo CNN)
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64, 64))

# 2. Converte a imagem para um array NumPy. Agora os dados da imagem são representados como valores de pixel.
test_image = image.img_to_array(test_image)

# 3. Adiciona uma nova dimensão ao array para simular um batch de tamanho 1.
# O modelo espera que as entradas estejam no formato (batch_size, height, width, channels).
test_image = np.expand_dims(test_image, axis=0)

# 4. Realiza a predição usando o modelo CNN treinado. O resultado será um array de probabilidades ou classes.
result = cnn.predict(test_image)

# 5. Imprime os índices das classes do conjunto de treino (necessário para mapear o índice para o nome da classe).
# Por exemplo, pode retornar {'cat': 0, 'dog': 1}.
print(train_set.class_indices)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
{'cats': 0, 'dogs': 1}


In [14]:
# 6. Interpreta o resultado da predição.
# Se o primeiro elemento no resultado (result[0][0]) for 1, a predição é 'dog'; caso contrário, é 'cat'.
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

# 7. Imprime a predição final, indicando se a imagem é de um cachorro ou um gato.
print(prediction)

cat
